In [ ]:
import { env } from './env.ts'

In [ ]:
import { TextLoader } from "langchain/document_loaders/fs/text";
const loader = new TextLoader('../dataset/qiu.txt');
const docs = await loader.load()

In [ ]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 500,
  chunkOverlap: 100,
});

const splitDocs = await splitter.splitDocuments(docs);

In [ ]:
import { OpenAIEmbeddings } from "@langchain/openai";

const embeddings = new OpenAIEmbeddings({
  configuration: {
    baseURL: env["QWEN_BASE_URL"],
    apiKey: env["QWEN_API_KEY"],
  },
  model: env["QWEN_EMBEDDINGS_MODEL"],
});

In [ ]:
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorStore = new MemoryVectorStore(embeddings);
for (let i = 0; i < splitDocs.length; i += 10) {
  await vectorStore.addDocuments(splitDocs.slice(i, i + 10));
}
const retriever = vectorStore.asRetriever(2);

In [ ]:
import { RunnableSequence } from "@langchain/core/runnables";
import { Document as IDocument } from "@langchain/core/documents";

const convertDocsToString = (docs: IDocument[]): string => {
  return docs.map((doc) => doc.pageContent).join("\n");
};

const contextRetrieverChain = RunnableSequence.from([
  (input) => input.question,
  retriever,
  convertDocsToString,
]);

In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from '@langchain/openai';
import { StringOutputParser } from "@langchain/core/output_parsers";

const SYSTEM_TEMPLATE = `
  你是一个熟读刘慈欣的《球状闪电》的终极原着党，精通根据作品原文详细解释和回答问题，你在回答时会引用作品原文。
  并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

  以下是原文中跟用户回答相关的内容：
  {context}

  现在，你需要基于原文，回答以下问题：
  {question}
`;

const systemPrompt = ChatPromptTemplate.fromTemplate(SYSTEM_TEMPLATE);

const llm = new ChatOpenAI({
  configuration: {
    baseURL: env["QWEN_BASE_URL"],
    apiKey: env["QWEN_API_KEY"],
  },
  model: env["QWEN_MODEL"],
});

const ragChain = RunnableSequence.from([
  {
    context: contextRetrieverChain,
    question: (input) => input.question,
  },
  systemPrompt,
  llm,
  new StringOutputParser(),
] as any);

In [ ]:
const res = await ragChain.invoke({
  question: "什么是球状闪电？",
});
res